# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [23]:
import json
'''average_rate(file) -> float
    >>> This function opens a json file on negotiated rates and reads the attributes of the allowed amounts to return the average rate as float
    >>> average_rate('/data/negotiated_rates.json')
    38.67
    '''
def average_rate(file):
    with open('/data/negotiated_rates.json') as jsonFile:
        negotiated_rates = json.load(jsonFile)
        totalOfAllowedAmounts = 0
        numberOfAllowedAmounts = 0

        for out_of_network in negotiated_rates["out_of_network"]:
            for allowed_amounts in out_of_network["allowed_amounts"]:
                for payments in allowed_amounts["payments"]:
                    amount = allowed_amounts["payments"]["allowed_amount"]
                    totalOfAllowedAmounts += amount
                    numberOfAllowedAmounts += 1
        totalOfAllowedAmounts = round(totalOfAllowedAmounts,2)
        averageOfAllowedAmount = totalOfAllowedAmounts / numberOfAllowedAmounts
        averageOfAllowedAmount = round(averageOfAllowedAmount,2)
    
    return averageOfAllowedAmount


In [24]:
average_rate('/data/negotiated_rates.json')

38.67

In [25]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [26]:
import json
'''get_rate(file, billing_code, service_code) -> float
    >>> This function opens a json file named negotiated rates and checks the parameters service code and billing code and compares them
    to the service code and the billing code from the file and returns the allowed amount as a float
    >>> get_rate('/data/negotiated_rates.json')
    
    '''
def get_rate(file, billing_code, service_code):
    with open('/data/negotiated_rates.json') as file:
        negotiated_rates = json.load(file)
        for out_of_network in negotiated_rates["out_of_network"]:
            allowed_amount = out_of_network['allowed_amounts']
            bil_code = out_of_network['billing_code']
            for code in allowed_amount:
                svc_code = code['service_code']
                if bil_code == billing_code and svc_code == service_code:
                    return code['payments']['allowed_amount']
        return None
    

In [27]:
get_rate('/data/negotiated_rates.json','G0283','11')

8.78

In [28]:
get_rate('/data/negotiated_rates.json','97140','11')

20.03

In [29]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [35]:
from datetime import datetime
'''get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date) -> float
    >>> This function opens a json file named negotiated rates and checks the parameters service code, billing code, patient age, the date of the visit and compares them
    to the parameters from the file and returns the adjusted rate as a float
    >>> get_adjusted_rate('/data/negotiated_rates.json')
    
    '''

def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    with open('/data/negotiated_rates.json') as file:
        negotiated_rates = json.load(file)
        day = datetime.strptime(visit_date, "%Y-%m-%d")
        weekday = day.strftime('%A')
        for out_of_network in negotiated_rates["out_of_network"]:
            allowed_amount = out_of_network['allowed_amounts']
            bil_code = out_of_network['billing_code']
            for code in allowed_amount:
                svc_code = code['service_code']
                if bil_code == billing_code and svc_code == service_code:
                    amount = code['payments']['allowed_amount']
                if weekday == 'Monday' and pat_age >= 65:
                    return round(amount * 0.5, 2)
                elif weekday == 'Monday':
                    return round(amount * 0.75, 2)
                elif pat_age >= 65:
                    return round(amount * 0.5, 2)
                else:
                    return round(amount, 2)
            return round(amount, 2)
    

In [31]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')

8.78

In [32]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')

4.39

In [33]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')

6.58

In [34]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [37]:
import json
import csv
from datetime import datetime
def summarize_reimbursement(visits, negotiated_rates)
"""
"""
by_month={}
by_hospital={}
CSV= "/data/visits.csv"
with open(CSV) as file:
    visits = csv.reader(file)
    for column in visits:
        if coloumn[0] == 'SYSTEM':
            continue
        hospital = column[1]
        patient_age= column[3]
        bil_code = column[4]
        svc_code = column[5]
        visit_date = column[6]
        adj_rate= get_adjusted_rate(file, bil_code, svc_code, int(patient_age, visit_date))
        if adj_rate is none:
                continue 
            if hospital in by_hospital.keys():
                by_hospital[hospital]=round(by_hospital[hospital])+adj_rate,2)
            else:
                by_hospital[hospital]=adj_rate
                
            visit_date= visit_date[:7]
            if visit_date in by_month.keys():
                by_month[visit_date]=round (by_month[visit_date] + adj_rate,2)
                
            else:
                by_month[visit_date]=adj_rate
                
        return by_month, by_hospital

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 23)

In [38]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

NameError: name 'summarize_reimbursement' is not defined

In [ ]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)